grok11


In [1]:
# Working almost Perfectly!  Just needs a collapsible trade log and a few more charts and metrics!
# 
# 
# 
# 
# runner_enhanced_v5.py – AlgoHaus Backtester v5.0
# FIXED: Trade log (trades_df) persistence issue for report generation.

import tkinter as tk
from tkinter import ttk, scrolledtext, messagebox, filedialog
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import webbrowser
import os
import tempfile
import inspect
import pathlib
import threading
import queue
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import re 
import logging

# Configure basic logging to see debug prints in console
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# ======================================================================
# 1. MARGIN AND PIP CALCULATIONS
# ======================================================================
class ForexCalculator:
    """Handle all forex calculations including margins, pip values, and position sizing"""
   
    # Leverage options
    LEVERAGE_OPTIONS = [1, 10, 20, 30, 50, 100, 200, 500]
   
    # Standard pip values (for 1 standard lot = 100,000 units)
    PIP_VALUES = {
        'EUR/USD': 0.0001, 'GBP/USD': 0.0001, 'USD/JPY': 0.01,
        'USD/CHF': 0.0001, 'USD/CAD': 0.0001, 'AUD/USD': 0.0001,
        'NZD/USD': 0.0001, 'EUR/JPY': 0.01, 'GBP/JPY': 0.01,
        'AUD/JPY': 0.01, 'EUR/GBP': 0.0001, 'EUR/CHF': 0.0001,
        'GBP/CHF': 0.0001, 'CHF/JPY': 0.01, 'CAD/JPY': 0.01
    }
    
    @staticmethod
    def get_max_units_per_100usd(leverage=50):
        """Calculates the maximum tradeable unit size for every $100 in the account."""
        return 100 * leverage 
   
    @staticmethod
    def calculate_pip_value_in_usd(pair, unit_size, current_price, conversion_rate=1.0):
        """Calculate pip value in USD for given pair and unit size."""
        pip_size = ForexCalculator.PIP_VALUES.get(pair, 0.0001) 
       
        if pair.endswith('/USD'):
            pip_value = pip_size * unit_size
        elif pair.startswith('USD/'):
            pip_value = (pip_size / current_price) * unit_size
        else:
            pip_value_in_base = pip_size * unit_size
            pip_value = pip_value_in_base * conversion_rate 
       
        return pip_value
   
    @staticmethod
    def calculate_margin_required(pair, unit_size, current_price, leverage, conversion_rate=1.0):
        """Calculate margin required for a position in USD."""
        if pair.endswith('/USD'):
            position_value = unit_size * current_price
        elif pair.startswith('USD/'):
            position_value = unit_size
        else:
            position_value = unit_size * conversion_rate 
       
        margin_required = position_value / leverage
        return margin_required
   
    @staticmethod
    def calculate_position_size(account_balance, risk_percent, stop_loss_pips, pair, current_price, conversion_rate=1.0):
        """Calculate optimal position size based on risk management"""
        risk_amount = account_balance * (risk_percent / 100)
        pip_value_per_unit = ForexCalculator.calculate_pip_value_in_usd(pair, 1, current_price, conversion_rate)
       
        if pip_value_per_unit > 0:
            position_size = risk_amount / (stop_loss_pips * pip_value_per_unit)
        else:
            position_size = 0
           
        return int(position_size) 

# ======================================================================
# 2. ENHANCED DATA LOADER
# ======================================================================
def get_pair_date_range(pair_name: str, base_folder: pathlib.Path):
    """Get the available date range for a pair without loading all data"""
    # (Implementation remains the same as previous version)
    try:
        pair_clean = pair_name.replace('/', '_')
        subfolder = base_folder / pair_clean
        csv_files = []
       
        if subfolder.is_dir():
            csv_files = list(subfolder.glob("*.csv"))
       
        if not csv_files:
            csv_files = [f for f in base_folder.glob("*.csv") if pair_clean.lower() in f.name.lower()]
       
        if not csv_files:
            return None, None
       
        csv_path = csv_files[0]
       
        # Quick sample to get date range
        df_dates = pd.read_csv(csv_path, usecols=[0], nrows=1000)
        if df_dates.empty:
            return None, None
           
        col_name = df_dates.columns[0]
        df_dates[col_name] = pd.to_datetime(df_dates[col_name], errors='coerce', utc=True)
        df_dates = df_dates.dropna()
       
        if df_dates.empty:
            return None, None
           
        first_date = df_dates[col_name].min()
       
        # Get last date efficiently
        try:
            file_size = csv_path.stat().st_size
            if file_size > 1000000:
                with open(csv_path, 'rb') as f:
                    f.seek(max(0, file_size - 100000))
                    f.readline()
                    tail_data = f.read().decode('utf-8', errors='ignore')
                   
                from io import StringIO
                df_tail = pd.read_csv(StringIO(tail_data), header=None, usecols=[0])
                df_tail[0] = pd.to_datetime(df_tail[0], errors='coerce', utc=True)
                df_tail = df_tail.dropna()
                last_date = df_tail[0].max() if not df_tail.empty else first_date
            else:
                df_all = pd.read_csv(csv_path, usecols=[0])
                df_all[col_name] = pd.to_datetime(df_all[col_name], errors='coerce', utc=True)
                df_all = df_all.dropna()
                last_date = df_all[col_name].max()
        except:
            last_date = first_date
       
        if pd.isna(first_date) or pd.isna(last_date):
            return None, None
           
        return first_date.date(), last_date.date()
    except Exception as e:
        logging.error(f"Error getting date range for {pair_name}: {e}")
        return None, None
        
def load_pair_data(pair_name: str, base_folder: pathlib.Path, start_date: datetime, end_date: datetime, timeframe: str):
    """Load and optimize data as DataFrame for fast backtesting."""
    # (Implementation remains the same as previous version)
    pair_clean = pair_name.replace('/', '_')
    subfolder = base_folder / pair_clean
    csv_files = []
   
    if subfolder.is_dir():
        csv_files = list(subfolder.glob("*.csv"))
   
    if not csv_files:
        csv_files = [f for f in base_folder.glob("*.csv") if pair_clean.lower() in f.name.lower()]
   
    if not csv_files:
        raise FileNotFoundError(f"No CSV found for {pair_name}")
   
    csv_path = csv_files[0]
   
    # Read sample to detect columns
    sample = pd.read_csv(csv_path, nrows=100)
    if sample.empty:
        raise ValueError("CSV is empty")
   
    # Column mapping
    cols_lower = [c.strip().lower() for c in sample.columns]
    col_map = {
        'datetime': ['datetime', 'date', 'time', 'timestamp', 'date_time', 'index', 'tick'],
        'open': ['open', 'o', 'bid', 'ask', 'price'],
        'high': ['high', 'h', 'max'],
        'low': ['low', 'l', 'min'],
        'close': ['close', 'c', 'last', 'price'],
        'volume': ['volume', 'vol', 'v', 'tick_volume', 'size']
    }
   
    rename = {}
    for target, aliases in col_map.items():
        for alias in aliases:
            if any(alias in col for col in cols_lower):
                orig = next(col for col in sample.columns if alias in col.lower())
                rename[orig] = target
                break
        else:
            if target != 'volume':
                raise KeyError(f"Column for '{target}' not found")
   
    # Load data
    dtypes = {}
    for col in rename.keys():
        if rename[col] != 'datetime':
            dtypes[col] = 'float32'
   
    df = pd.read_csv(csv_path, usecols=list(rename.keys()), dtype=dtypes)
    df = df.rename(columns=rename)
   
    if 'volume' not in df.columns:
        df['volume'] = 1000
   
    # Parse dates
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce', utc=True)
    df = df.dropna(subset=['datetime'])
    df['datetime'] = df['datetime'].dt.tz_localize(None)
    
    # Handle duplicate datetimes
    df = df.drop_duplicates(subset='datetime', keep='first')
    df = df.sort_values('datetime')
   
    # Get actual range
    actual_start = df['datetime'].min().date()
    actual_end = df['datetime'].max().date()
   
    # Filter to date range
    df = df.set_index('datetime')
    user_start = max(pd.Timestamp(start_date.date()), df.index.min())
    user_end = min(pd.Timestamp(end_date.date()) + pd.Timedelta(hours=23, minutes=59, seconds=59), df.index.max())
    df = df.loc[user_start:user_end].copy()
   
    if df.empty:
        raise ValueError(f"No data in range {start_date.date()} to {end_date.date()}")
   
    # Resample if needed
    if timeframe != '1min':
        rule = {'5min': '5T', '15min': '15T', '1hr': '1H', '1Day': '1D'}.get(timeframe, '1T')
        df = df.resample(rule).agg({
            'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'
        }).dropna()
   
    # Add daily data WITHOUT LOOK-AHEAD BIAS
    df = df.reset_index()
    df['date'] = df['datetime'].dt.date
   
    # Calculate previous day's values (no look-ahead)
    daily = df.groupby('date').agg({
        'high': 'max', 'low': 'min', 'close': 'last'
    })
    daily.columns = ['day_high', 'day_low', 'day_close']
   
    # SHIFT to avoid look-ahead bias
    daily['prev_high'] = daily['day_high'].shift(1)
    daily['prev_low'] = daily['day_low'].shift(1)
    daily['prev_close'] = daily['day_close'].shift(1)
   
    daily = daily.reset_index()
    df = pd.merge(df, daily[['date', 'prev_high', 'prev_low', 'prev_close']], on='date', how='left')
    df[['prev_high', 'prev_low', 'prev_close']] = df[['prev_high', 'prev_low', 'prev_close']].ffill()
   
    return df, actual_start, actual_end
    
# ======================================================================
# 3. TRADING STRATEGIES
# ======================================================================
class TradingStrategies:
    @staticmethod
    def opening_range_strategy(df, sl_pips, tp_pips, pip_value):
        """Opening Range Strategy - NO LOOK-AHEAD BIAS"""
        df = df.copy()
        trades = []
       
        # Group by day for daily opening range
        for date in df['date'].unique():
            day_data = df[df['date'] == date].reset_index(drop=True)
            if len(day_data) < 31: 
                continue
           
            # First 30 minutes (assuming 1-min data)
            opening_range = day_data.iloc[:30]
            or_high = opening_range['high'].max()
            or_low = opening_range['low'].min()
            or_mid = (or_high + or_low) / 2
           
            # Entry at bar 31 (after 30-min range)
            if len(day_data) > 30:
                entry_bar = day_data.iloc[30]
                signal = 'BUY' if entry_bar['close'] > or_mid else 'SELL'
               
                trades.append({
                    'datetime': entry_bar['datetime'],
                    'entry_price': entry_bar['close'],
                    'signal': signal,
                    'stop_loss': sl_pips * pip_value,
                    'take_profit': tp_pips * pip_value,
                    'day_data': day_data[31:].reset_index(drop=True) # Only future bars
                })
       
        return trades

    @staticmethod
    def vwap_crossover_strategy(df, sl_pips, tp_pips, pip_value):
        """VWAP Crossover - NO LOOK-AHEAD BIAS"""
        df = df.copy()
       
        # Calculate VWAP properly
        df['tpv'] = df['volume'] * (df['high'] + df['low'] + df['close']) / 3
        df['cumvol'] = df.groupby('date')['volume'].cumsum()
        df['cumtpv'] = df.groupby('date')['tpv'].cumsum()
        df['vwap'] = df['cumtpv'] / df['cumvol']
       
        # Detect crossovers using PREVIOUS bar's VWAP
        df['prev_close'] = df['close'].shift(1)
        df['prev_vwap'] = df['vwap'].shift(1)
       
        # Signal when price crosses VWAP
        df['signal'] = np.where(
            (df['prev_close'] <= df['prev_vwap']) & (df['close'] > df['vwap']), 'BUY',
            np.where((df['prev_close'] >= df['prev_vwap']) & (df['close'] < df['vwap']), 'SELL', None)
        )
       
        # Get trade entries
        entries = df[df['signal'].notna()].copy()
        trades = []
       
        for idx, row in entries.iterrows():
            remaining_data = df[df.index > idx].reset_index(drop=True)
            if len(remaining_data) > 0:
                trades.append({
                    'datetime': row['datetime'],
                    'entry_price': row['close'],
                    'signal': row['signal'],
                    'stop_loss': sl_pips * pip_value,
                    'take_profit': tp_pips * pip_value,
                    'day_data': remaining_data
                })
       
        return trades

    @staticmethod
    def pivot_point_reversal_strategy(df, sl_pips, tp_pips, pip_value):
        """Pivot Point Reversal - NO LOOK-AHEAD BIAS"""
        df = df.copy()
       
        # Calculate pivots using PREVIOUS day's data
        df['pivot'] = (df['prev_high'] + df['prev_low'] + df['prev_close']) / 3
        df['r1'] = 2 * df['pivot'] - df['prev_low']
        df['s1'] = 2 * df['pivot'] - df['prev_high']
       
        # Remove first day (no previous data)
        df = df[df['prev_high'].notna()].copy()
       
        trades = []
        # Define a small tolerance zone (e.g., 5 pips)
        bounce_zone = 5 * pip_value
       
        # Check each bar for pivot bounces
        for i in range(1, len(df)): # Start from 1 to use previous bar
            current = df.iloc[i]
           
            # Buy signal at support S1
            if abs(current['close'] - current['s1']) < bounce_zone:
                remaining = df.iloc[i+1:].reset_index(drop=True)
                if len(remaining) > 0:
                    trades.append({
                        'datetime': current['datetime'],
                        'entry_price': current['close'],
                        'signal': 'BUY',
                        'stop_loss': sl_pips * pip_value,
                        'take_profit': tp_pips * pip_value,
                        'day_data': remaining
                    })
           
            # Sell signal at resistance R1
            elif abs(current['close'] - current['r1']) < bounce_zone:
                remaining = df.iloc[i+1:].reset_index(drop=True)
                if len(remaining) > 0:
                    trades.append({
                        'datetime': current['datetime'],
                        'entry_price': current['close'],
                        'signal': 'SELL',
                        'stop_loss': sl_pips * pip_value,
                        'take_profit': tp_pips * pip_value,
                        'day_data': remaining
                    })
       
        return trades
        
# ======================================================================
# 4. ENHANCED BACKTEST ENGINE
# ======================================================================
class EnhancedBacktester:
    def __init__(self, df, initial_balance=10000, unit_size=10000, pip_value=0.0001, leverage=50):
        self.df = df
        self.initial_balance = initial_balance
        self.unit_size = unit_size
        self.pip_value = pip_value
        self.leverage = leverage
        self.results = None
        self.balance_history = []
        
    def get_conversion_rate(self, pair_name, bar_data):
        """Retrieves the necessary Base/USD conversion rate for cross-pairs."""
        # For simplicity in a single-pair backtester, we assume 1.0
        # A real system would require a second data feed for the Base/USD pair
        return 1.0 
            
    def run_backtest(self, strategy_func, sl_pips, tp_pips, pair_name):
        """Run backtest with proper margin and P&L calculations"""
        logging.info(f"Dataframe size: {len(self.df)} bars.")
        
        trades = strategy_func(self.df, sl_pips, tp_pips, self.pip_value)
        
        logging.info(f"Strategy generated: {len(trades)} potential trades.")
        if not trades:
            self.results = pd.DataFrame()
            return "No trades generated.", {}
            
        results = []
        current_balance = self.initial_balance
        trade_number = 1
       
        for t in trades:
            entry_price = t['entry_price']
            signal = t['signal']
            bars = t['day_data']
            
            # --- Conversion and Margin Calculation ---
            conversion_rate = self.get_conversion_rate(pair_name, {'datetime': t['datetime'], 'close': entry_price})
           
            margin_required = ForexCalculator.calculate_margin_required(
                pair_name, self.unit_size, entry_price, self.leverage, conversion_rate
            )
           
            # Skip trade if insufficient margin (Risk Guardrail)
            if margin_required > current_balance * 0.5: # Max 50% margin usage
                continue
           
            pip_value_usd = ForexCalculator.calculate_pip_value_in_usd(
                pair_name, self.unit_size, entry_price, conversion_rate
            )
            # --- End Calculations ---
           
            # Set stop loss and take profit levels
            if signal == 'BUY':
                stop_level = entry_price - (sl_pips * self.pip_value)
                take_level = entry_price + (tp_pips * self.pip_value)
            else:
                stop_level = entry_price + (sl_pips * self.pip_value)
                take_level = entry_price - (tp_pips * self.pip_value)
           
            # Find exit point
            exit_idx = len(bars) - 1
            exit_reason = 'Timeout'
            exit_price = bars.iloc[-1]['close'] if len(bars) > 0 else entry_price
           
            for i, bar in bars.iterrows():
                if signal == 'BUY':
                    if bar['low'] <= stop_level:
                        exit_idx = i
                        exit_price = stop_level
                        exit_reason = 'SL'
                        break
                    elif bar['high'] >= take_level:
                        exit_idx = i
                        exit_price = take_level
                        exit_reason = 'TP'
                        break
                else: # SELL
                    if bar['high'] >= stop_level:
                        exit_idx = i
                        exit_price = stop_level
                        exit_reason = 'SL'
                        break
                    elif bar['low'] <= take_level:
                        exit_idx = i
                        exit_price = take_level
                        exit_reason = 'TP'
                        break
           
            # Calculate P&L
            if signal == 'BUY':
                pips_pnl = (exit_price - entry_price) / self.pip_value
            else:
                pips_pnl = (entry_price - exit_price) / self.pip_value
           
            # Monetary P&L calculation
            monetary_pnl = pips_pnl * pip_value_usd
           
            # Calculate time in trade
            entry_time = t['datetime']
            if exit_idx < len(bars):
                exit_time = bars.iloc[exit_idx]['datetime']
            else:
                exit_time = bars.iloc[-1]['datetime'] if len(bars) > 0 else entry_time
           
            time_in_trade = exit_time - entry_time
            hours_in_trade = time_in_trade.total_seconds() / 3600
           
            # Update balance
            current_balance += monetary_pnl
           
            results.append({
                'trade_number': f"{trade_number:05d}",
                'entry_time': entry_time,
                'exit_time': exit_time,
                'time_in_trade_hours': round(hours_in_trade, 2),
                'signal': signal,
                'entry_price': entry_price,
                'exit_price': exit_price,
                'exit_reason': exit_reason,
                'pips_pnl': round(pips_pnl, 2),
                'monetary_pnl': round(monetary_pnl, 2),
                'unit_size': self.unit_size,
                'margin_used': round(margin_required, 2),
                'balance': round(current_balance, 2),
                'pip_value_usd': round(pip_value_usd, 4)
            })
           
            trade_number += 1
            self.balance_history.append(current_balance)
       
        self.results = pd.DataFrame(results)
       
        # Calculate metrics
        if not self.results.empty:
            total_pnl = self.results['monetary_pnl'].sum()
            total_pips = self.results['pips_pnl'].sum()
            win_rate = (self.results['pips_pnl'] > 0).mean() * 100
           
            summary = (f"TRADES: {len(self.results)}\n"
                      f"WIN RATE: {win_rate:.1f}%\n"
                      f"P&L: ${total_pnl:,.2f}\n"
                      f"PIPS: {total_pips:,.1f}\n"
                      f"FINAL BALANCE: ${current_balance:,.2f}")
           
            metrics = QuantStatsMetrics.calculate_all_metrics(
                self.results, self.initial_balance
            )
        else:
            summary = "No trades executed"
            metrics = {}
       
        return summary, metrics
        
# ======================================================================
# 5. QUANTSTATS-STYLE METRICS
# ======================================================================
class QuantStatsMetrics:
    # (Implementation remains the same as previous version)
    @staticmethod
    def calculate_all_metrics(trades_df, initial_balance):
        """Calculate comprehensive QuantStats-style metrics"""
        if trades_df.empty:
            return {}
       
        # Basic metrics
        total_trades = len(trades_df)
        winning_trades = (trades_df['monetary_pnl'] > 0).sum()
        losing_trades = (trades_df['monetary_pnl'] < 0).sum()
        win_rate = (winning_trades / total_trades * 100) if total_trades > 0 else 0
       
        # P&L metrics
        total_pnl = trades_df['monetary_pnl'].sum()
        total_pips = trades_df['pips_pnl'].sum()
        avg_win = trades_df[trades_df['monetary_pnl'] > 0]['monetary_pnl'].mean() if winning_trades > 0 else 0
        avg_loss = abs(trades_df[trades_df['monetary_pnl'] < 0]['monetary_pnl'].mean()) if losing_trades > 0 else 0
        avg_win_pips = trades_df[trades_df['pips_pnl'] > 0]['pips_pnl'].mean() if winning_trades > 0 else 0
        avg_loss_pips = abs(trades_df[trades_df['pips_pnl'] < 0]['pips_pnl'].mean()) if losing_trades > 0 else 0
       
        # Best and worst trades
        best_trade = trades_df['monetary_pnl'].max()
        worst_trade = trades_df['monetary_pnl'].min()
        best_trade_pips = trades_df['pips_pnl'].max()
        worst_trade_pips = trades_df['pips_pnl'].min()
       
        # Profit factor
        gross_profit = trades_df[trades_df['monetary_pnl'] > 0]['monetary_pnl'].sum()
        gross_loss = abs(trades_df[trades_df['monetary_pnl'] < 0]['monetary_pnl'].sum())
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')
       
        # Returns
        final_balance = trades_df['balance'].iloc[-1] if not trades_df.empty else initial_balance
        total_return = ((final_balance - initial_balance) / initial_balance) * 100
       
        # Create equity curve
        equity_curve = initial_balance + trades_df['monetary_pnl'].cumsum()
       
        # Drawdown calculations
        cummax = equity_curve.expanding().max()
        drawdown = (equity_curve - cummax) / cummax
        max_drawdown_pct = drawdown.min() * 100
       
        # Time-based metrics
        start_date = trades_df['entry_time'].min()
        end_date = trades_df['exit_time'].max()
        trading_days = (end_date - start_date).days if pd.notna(start_date) and pd.notna(end_date) else 1
       
        # Annualized return
        years = trading_days / 365 if trading_days > 0 else 1
        annualized_return = ((final_balance / initial_balance) ** (1/years) - 1) * 100 if years > 0 else 0
       
        # Sharpe Ratio (simplified - using daily returns)
        if len(trades_df) > 1:
            daily_returns = trades_df.groupby(trades_df['entry_time'].dt.date)['monetary_pnl'].sum()
            daily_returns_pct = daily_returns / initial_balance
            sharpe = (daily_returns_pct.mean() * 252) / (daily_returns_pct.std() * np.sqrt(252)) if daily_returns_pct.std() > 0 else 0
        else:
            sharpe = 0
       
        # Sortino Ratio (using downside returns only)
        if len(trades_df) > 1:
            negative_returns = daily_returns_pct[daily_returns_pct < 0]
            downside_std = negative_returns.std() if len(negative_returns) > 0 else 0
            sortino = (daily_returns_pct.mean() * 252) / (downside_std * np.sqrt(252)) if downside_std > 0 else 0
        else:
            sortino = 0
       
        # Average time in trade
        avg_time_in_trade = trades_df['time_in_trade_hours'].mean() if 'time_in_trade_hours' in trades_df else 0
       
        # Margin metrics
        avg_margin = trades_df['margin_used'].mean() if 'margin_used' in trades_df else 0
       
        # Average trade PnL
        avg_trade_pnl = total_pnl / total_trades if total_trades > 0 else 0
       
        return {
            # Trade Statistics
            'total_trades': total_trades,
            'winning_trades': winning_trades,
            'losing_trades': losing_trades,
            'win_rate_%': round(win_rate, 2),
           
            # P&L Metrics
            'total_pnl_$': round(total_pnl, 2),
            'total_pips': round(total_pips, 1),
            'avg_win_$': round(avg_win, 2),
            'avg_loss_$': round(avg_loss, 2),
            'avg_win_pips': round(avg_win_pips, 1),
            'avg_loss_pips': round(avg_loss_pips, 1),
            'avg_trade_pnl_$': round(avg_trade_pnl, 2),
           
            # Best/Worst
            'best_trade_$': round(best_trade, 2),
            'worst_trade_$': round(worst_trade, 2),
            'best_trade_pips': round(best_trade_pips, 1),
            'worst_trade_pips': round(worst_trade_pips, 1),
           
            # Risk Metrics
            'profit_factor': round(profit_factor, 2),
            'max_drawdown_%': round(max_drawdown_pct, 2),
            'sharpe_ratio': round(sharpe, 2),
            'sortino_ratio': round(sortino, 2),
           
            # Return Metrics
            'total_return_%': round(total_return, 2),
            'annualized_return_%': round(annualized_return, 2),
           
            # Other Metrics
            'avg_time_in_trade_hours': round(avg_time_in_trade, 2),
            'avg_margin_used_$': round(avg_margin, 2),
            'final_balance_$': round(final_balance, 2)
        }

# ======================================================================
# 6. ADVANCED HTML REPORT GENERATOR
# ======================================================================
class AdvancedHTMLReportGenerator:
    # (Implementation remains the same as previous version)
    @staticmethod
    def generate_dashboard_report(metrics, trades_df, strategy, timeframe, pair, initial_balance, unit_size, df=None):
        """Generate comprehensive HTML report with dashboard elements"""
       
        # Create timestamp for filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
       
        # Get strategy source code
        strategy_code = AdvancedHTMLReportGenerator.get_strategy_code(strategy)
       
        # Generate the comparison chart first (for top placement)
        comparison_chart_html = AdvancedHTMLReportGenerator.generate_comparison_chart(trades_df, initial_balance, df)
       
        # Generate other plots
        plots_html = AdvancedHTMLReportGenerator.generate_plots(trades_df, initial_balance, df)
       
        # AI-style analysis text
        analysis = AdvancedHTMLReportGenerator.generate_ai_analysis(metrics, trades_df, df)
       
        html = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>AlgoHaus Backtest Report - {pair} {strategy.__name__}</title>
            <meta charset="utf-8">
            <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
            <style>
                /* CSS styles truncated for brevity */
                body {{
                    font-family: 'Consolas', 'Courier New', monospace;
                    background: #000000;
                    color: #ffffff;
                    margin: 0;
                    padding: 0;
                }}
                .header {{ background: #000000; color: #ffffff; padding: 20px; border-bottom: 1px solid #333; }}
                .header h1 {{ margin: 0; font-size: 18px; font-weight: bold; font-family: 'Consolas', monospace; }}
                .primary-chart {{ background: #0a0a0a; padding: 20px; border-bottom: 2px solid #1976D2; }}
                .primary-chart h2 {{ color: #1976D2; font-size: 20px; margin: 0 0 20px 0; font-weight: bold; font-family: 'Consolas', monospace; }}
                .strategy-code-section {{ background: #0a0a0a; padding: 20px; border-bottom: 1px solid #333; }}
                .strategy-code-button {{ background: #1976D2; color: white; border: none; padding: 10px 20px; font-size: 14px; font-family: 'Consolas', monospace; cursor: pointer; border-radius: 4px; margin-bottom: 15px; }}
                .strategy-code-content {{ display: none; background: #1a1a1a; border: 1px solid #333; border-radius: 5px; padding: 20px; overflow-x: auto; }}
                .strategy-code-content.show {{ display: block; }}
                .strategy-code-content pre {{ margin: 0; color: #e0e0e0; font-family: 'Consolas', 'Courier New', monospace; font-size: 12px; line-height: 1.4; white-space: pre; }}
                .code-keyword {{ color: #569CD6; }}
                .code-string {{ color: #CE9178; }}
                .code-comment {{ color: #6A9955; }}
                .code-function {{ color: #DCDCAA; }}
                .code-number {{ color: #B5CEA8; }}
                .metrics-section {{ background: #000000; color: #ffffff; padding: 20px; font-family: 'Consolas', monospace; border-bottom: 1px solid #333; }}
                .metrics-section h2 {{ font-size: 16px; margin: 0 0 15px 0; font-weight: bold; }}
                .metrics-table {{ font-size: 12px; line-height: 1.4; }}
                .trades-section {{ background: #0a0a0a; padding: 20px; border-bottom: 1px solid #333; }}
                .trades-section h2 {{ color: #ffffff; font-size: 16px; margin: 0 0 15px 0; font-weight: bold; font-family: 'Consolas', monospace; }}
                table {{ width: 100%; border-collapse: collapse; font-family: 'Consolas', monospace; font-size: 12px; }}
                th {{ background: #1a1a1a; color: #ffffff; padding: 8px 4px; text-align: left; font-weight: normal; border-bottom: 1px solid #333; white-space: nowrap; }}
                td {{ padding: 6px 4px; border-bottom: 1px solid #1a1a1a; white-space: nowrap; }}
                tr:nth-child(even) {{ background-color: #0d0d0d; }}
                tr:hover {{ background-color: #1a1a1a; }}
                .positive {{ color: #4CAF50; }}
                .negative {{ color: #F44336; }}
                .neutral {{ color: #E0E0E0; }}
                .key-value-pair {{ display: flex; justify-content: space-between; border-bottom: 1px dotted #333; padding: 4px 0; }}
                .key {{ color: #B5CEA8; }}
                .value {{ color: #DCDCAA; font-weight: bold; }}
                .analysis-section {{ background: #000000; padding: 20px; border-bottom: 1px solid #333; }}
                .analysis-section h2 {{ font-size: 16px; color: #1976D2; margin: 0 0 15px 0; font-weight: bold; }}
                .analysis-text {{ line-height: 1.6; font-size: 14px; color: #E0E0E0; white-space: pre-wrap; }}
                .container {{ display: grid; grid-template-columns: 2fr 1fr; gap: 20px; padding: 20px; }}
                .chart-container {{ grid-column: 1 / 2; }}
                .metrics-summary {{ grid-column: 2 / 3; background: #0a0a0a; padding: 15px; border-radius: 5px; border: 1px solid #1a1a1a; }}
                .main-content {{ display: flex; flex-direction: column; gap: 20px; }}
                .plot-grid {{ display: grid; grid-template-columns: 1fr 1fr; gap: 20px; padding: 20px; }}
            </style>
        </head>
        <body>
        <div class="header">
            <h1>AlgoHaus Backtest Report v5.0</h1>
            <p>Pair: {pair} | Strategy: {strategy.__name__} | Timeframe: {timeframe} | Date: {timestamp}</p>
        </div>
        
        <div class="primary-chart">
            <h2>Equity Curve and Price Action Comparison</h2>
            {comparison_chart_html}
        </div>
        
        <div class="container">
            <div class="main-content">
                <div class="metrics-section">
                    <h2>📊 Quantitative Metrics Summary</h2>
                    <div class="metrics-summary">
                        {AdvancedHTMLReportGenerator.generate_metrics_table(metrics, initial_balance)}
                    </div>
                </div>
                
                <div class="analysis-section">
                    <h2>🧠 AI Analysis Report</h2>
                    <pre class="analysis-text">{analysis}</pre>
                </div>
                
                <div class="strategy-code-section">
                    <button class="strategy-code-button" onclick="document.getElementById('codeContent').classList.toggle('show')">
                        Show Strategy Source Code
                    </button>
                    <div id="codeContent" class="strategy-code-content">
                        <pre>{AdvancedHTMLReportGenerator.highlight_code(strategy_code)}</pre>
                    </div>
                </div>
            </div>
            
            <div class="metrics-summary">
                <h2>📈 Additional Charts</h2>
                {plots_html}
            </div>
        </div>
        
        <div class="trades-section">
            <h2>📝 Trade Log ({len(trades_df)} Trades)</h2>
            {AdvancedHTMLReportGenerator.generate_trades_table(trades_df)}
        </div>
        
        <script>
            // Add script for toggling the code block if needed
        </script>
        </body>
        </html>
        """
        
        # Save to temp file
        temp_dir = tempfile.gettempdir()
        report_path = os.path.join(temp_dir, f"backtest_report_{pair.replace('/', '_')}_{strategy.__name__}_{timestamp}.html")
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(html)
            
        return report_path

    @staticmethod
    def generate_metrics_table(metrics, initial_balance):
        """Generates the key metrics table HTML structure"""
        # (Implementation remains the same as previous version)
        html = f"""
        <div class="metrics-table">
            <div class="key-value-pair"><span class="key">Initial Balance</span><span class="value neutral">${initial_balance:,.2f}</span></div>
            <div class="key-value-pair"><span class="key">Final Balance</span><span class="value {'positive' if metrics.get('final_balance_$', 0) >= initial_balance else 'negative'}>${metrics.get('final_balance_$', 0):,.2f}</span></div>
            <div class="key-value-pair"><span class="key">Total Return</span><span class="value {'positive' if metrics.get('total_return_%', 0) >= 0 else 'negative'}>{metrics.get('total_return_%', 0):.2f}%</span></div>
            <div class="key-value-pair"><span class="key">Annualized Return</span><span class="value {'positive' if metrics.get('annualized_return_%', 0) >= 0 else 'negative'}>{metrics.get('annualized_return_%', 0):.2f}%</span></div>
            <div class="key-value-pair">---</div>
            <div class="key-value-pair"><span class="key">Total Trades</span><span class="value neutral">{metrics.get('total_trades', 0)}</span></div>
            <div class="key-value-pair"><span class="key">Win Rate</span><span class="value {'positive' if metrics.get('win_rate_%', 0) >= 50 else 'neutral'}>{metrics.get('win_rate_%', 0):.2f}%</span></div>
            <div class="key-value-pair"><span class="key">Profit Factor</span><span class="value {'positive' if metrics.get('profit_factor', 0) >= 1 else 'negative'}>{metrics.get('profit_factor', 0):.2f}</span></div>
            <div class="key-value-pair"><span class="key">Max Drawdown</span><span class="value negative">{metrics.get('max_drawdown_%', 0):.2f}%</span></div>
            <div class="key-value-pair">---</div>
            <div class="key-value-pair"><span class="key">Total P&L ($)</span><span class="value {'positive' if metrics.get('total_pnl_$', 0) >= 0 else 'negative'}>${metrics.get('total_pnl_$', 0):,.2f}</span></div>
            <div class="key-value-pair"><span class="key">Total Pips</span><span class="value {'positive' if metrics.get('total_pips', 0) >= 0 else 'negative'}>{metrics.get('total_pips', 0):,.1f}</span></div>
            <div class="key-value-pair"><span class="key">Sharpe Ratio</span><span class="value {'positive' if metrics.get('sharpe_ratio', 0) >= 0.5 else 'neutral'}>{metrics.get('sharpe_ratio', 0):.2f}</span></div>
            <div class="key-value-pair"><span class="key">Avg Win/Loss Ratio</span><span class="value {'positive' if metrics.get('avg_win_$', 0) / (metrics.get('avg_loss_$', 1) if metrics.get('avg_loss_$', 1) != 0 else 1) > 1 else 'neutral'}>{metrics.get('avg_win_$', 0):,.2f} / {metrics.get('avg_loss_$', 0):,.2f}</span></div>
            <div class="key-value-pair"><span class="key">Avg Margin Used ($)</span><span class="value neutral">${metrics.get('avg_margin_used_$', 0):,.2f}</span></div>
        </div>
        """
        return html

    @staticmethod
    def generate_trades_table(trades_df):
        """Generates the trade log table HTML structure"""
        if trades_df.empty:
            return "<p>No trades executed in the backtest.</p>"
            
        trades_df_html = trades_df.to_html(
            classes='trades-table',
            index=False,
            justify='left',
            float_format=lambda x: f'{x:,.4f}' if abs(x) < 1000 and abs(x) > 0.1 else f'{x:,.2f}',
            columns=['trade_number', 'entry_time', 'exit_time', 'time_in_trade_hours', 'signal', 'entry_price', 'exit_price', 'pips_pnl', 'monetary_pnl', 'exit_reason', 'balance']
        )
        return trades_df_html.replace('<td>', '<td class="neutral">')

    @staticmethod
    def generate_comparison_chart(trades_df, initial_balance, df):
        """Generates the combined equity curve and price chart using Plotly"""
        if trades_df.empty or df is None:
            return "<p>Not enough data or trades to generate the Comparison Chart.</p>"

        equity_curve = initial_balance + trades_df['monetary_pnl'].cumsum()
        
        max_eq_date = trades_df['exit_time'].max()
        df_plot = df[df['datetime'] <= max_eq_date]
        
        if df_plot.empty:
            return "<p>Not enough price data after trades to generate the Comparison Chart.</p>"
            
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                            vertical_spacing=0.1, 
                            row_heights=[0.7, 0.3])

        # 1. Price Chart (Top Panel)
        fig.add_trace(go.Candlestick(x=df_plot['datetime'],
                                     open=df_plot['open'],
                                     high=df_plot['high'],
                                     low=df_plot['low'],
                                     close=df_plot['close'],
                                     name='Price',
                                     increasing_line_color='#4CAF50', 
                                     decreasing_line_color='#F44336'), 
                      row=1, col=1)

        # 2. Equity Curve (Bottom Panel)
        fig.add_trace(go.Scatter(x=trades_df['exit_time'], y=equity_curve,
                                 mode='lines',
                                 name='Equity Curve',
                                 line=dict(color='#1976D2', width=2)), 
                      row=2, col=1)
                      
        # Add Trade markers
        winners = trades_df[trades_df['monetary_pnl'] > 0]
        losers = trades_df[trades_df['monetary_pnl'] < 0]

        fig.add_trace(go.Scatter(x=winners['exit_time'], y=winners['exit_price'],
                                 mode='markers', name='Win',
                                 marker=dict(symbol='triangle-up', size=8, color='#4CAF50'),
                                 hovertext='Win: $' + winners['monetary_pnl'].round(2).astype(str),
                                 showlegend=True),
                      row=1, col=1)
                      
        fig.add_trace(go.Scatter(x=losers['exit_time'], y=losers['exit_price'],
                                 mode='markers', name='Loss',
                                 marker=dict(symbol='triangle-down', size=8, color='#F44336'),
                                 hovertext='Loss: $' + losers['monetary_pnl'].round(2).astype(str),
                                 showlegend=True),
                      row=1, col=1)

        fig.update_layout(
            template="plotly_dark",
            title_text=f"Price Action and Equity Curve",
            xaxis_rangeslider_visible=False,
            height=600,
            showlegend=False
        )

        fig.update_yaxes(title_text="Price", row=1, col=1)
        fig.update_yaxes(title_text="Balance ($)", row=2, col=1)

        return fig.to_html(full_html=False, include_plotlyjs='cdn')

    @staticmethod
    def generate_plots(trades_df, initial_balance, df):
        """Generates additional metrics plots (Drawdown and P&L by Hour/Day)"""
        if trades_df.empty:
            return "<p>Not enough trades to generate plots.</p>"
            
        html = ""
        
        # 1. Drawdown Chart
        equity_curve = initial_balance + trades_df['monetary_pnl'].cumsum()
        cummax = equity_curve.expanding().max()
        drawdown = (equity_curve - cummax) / cummax
        
        fig_dd = go.Figure(data=[
            go.Scatter(x=trades_df['exit_time'], y=drawdown * 100,
                       mode='lines', line=dict(color='#F44336', width=2),
                       fill='tozeroy', fillcolor='rgba(244, 67, 54, 0.3)')
        ])
        fig_dd.update_layout(template="plotly_dark", 
                             title="Max Drawdown (%)", 
                             yaxis_title="Drawdown (%)", 
                             height=300, margin=dict(t=30, b=30))
        html += fig_dd.to_html(full_html=False, include_plotlyjs=False) + "<br>"

        # 2. P&L by Day of Week
        trades_df['day_of_week'] = trades_df['entry_time'].dt.day_name()
        daily_pnl = trades_df.groupby('day_of_week')['monetary_pnl'].sum().reindex([
            'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'], fill_value=0)
            
        fig_day = px.bar(daily_pnl, 
                         x=daily_pnl.index, 
                         y=daily_pnl.values, 
                         title="P&L by Day of Week ($)")
        fig_day.update_traces(marker_color=['#4CAF50' if x > 0 else '#F44336' for x in daily_pnl.values])
        fig_day.update_layout(template="plotly_dark", height=300, margin=dict(t=30, b=30))
        html += fig_day.to_html(full_html=False, include_plotlyjs=False)
        
        return html

    @staticmethod
    def generate_ai_analysis(metrics, trades_df, df):
        """Generate a brief analysis based on metrics"""
        # (Implementation remains the same as previous version)
        pnl = metrics.get('total_pnl_$', 0)
        win_rate = metrics.get('win_rate_%', 0)
        profit_factor = metrics.get('profit_factor', 0)
        max_dd = metrics.get('max_drawdown_%', 0)
        total_trades = metrics.get('total_trades', 0)

        analysis = "--- SYSTEM REPORT ---\n"
        
        if total_trades == 0:
            analysis += "ANALYSIS: No trades were executed. Check data quality, time range, margin requirements, or strategy logic.\n"
        elif pnl > 0:
            analysis += f"ANALYSIS: The strategy achieved a net profit of ${pnl:,.2f} over {total_trades} trades.\n"
            analysis += f"PERFORMANCE: Win Rate is {win_rate:.2f}%, indicating a decent hit rate. The Profit Factor ({profit_factor:.2f}) is above 1.0, signifying gross profit exceeds gross loss.\n"
            analysis += f"RISK: The Max Drawdown of {max_dd:.2f}% suggests manageable capital exposure, but tighter stop-loss management may be beneficial.\n"
        else:
            analysis += f"ANALYSIS: The strategy resulted in a net loss of ${abs(pnl):,.2f} over {total_trades} trades.\n"
            analysis += f"PERFORMANCE: The Profit Factor ({profit_factor:.2f}) is below 1.0. Revisit the strategy entry/exit logic or adjust the R:R ratio.\n"
            analysis += f"RISK: The Max Drawdown of {max_dd:.2f}% confirms the capital risk incurred during the losing period.\n"
            
        analysis += f"UNIT SIZE: Max recommended unit size per $100 on 50x leverage is {ForexCalculator.get_max_units_per_100usd(50):,} units.\n"
        analysis += f"AVG MARGIN: Average Margin Used: ${metrics.get('avg_margin_used_$', 0):,.2f}.\n"
        
        return analysis

    @staticmethod
    def get_strategy_code(strategy):
        """Retrieves the source code of the strategy function"""
        try:
            if hasattr(TradingStrategies, strategy.__name__):
                source = inspect.getsource(strategy)
                return source
            return "Source code not found."
        except Exception as e:
            return f"Error retrieving source code: {e}"

    @staticmethod
    def highlight_code(code):
        """Simple syntax highlighting for HTML report"""
        # (Implementation remains the same as previous version)
        keywords = ['def', 'return', 'if', 'elif', 'else', 'for', 'in', 'while', 'import', 'from', 'class', 'self', 'and', 'or', 'not', 'try', 'except', 'break', 'continue', 'with', 'as', 'lambda', 'None', 'True', 'False']
        
        def replace_with_span(match):
            word = match.group(0)
            if word in keywords:
                return f'<span class="code-keyword">{word}</span>'
            elif word.startswith('"') or word.startswith("'"):
                return f'<span class="code-string">\1</span>'
            elif word.isdigit() or ('.' in word and word.replace('.', '').isdigit()):
                return f'<span class="code-number">{word}</span>'
            return word

        import re
        code = code.replace('<', '&lt;').replace('>', '&gt;')
        
        code = re.sub(r"(\".*?\"|\'.*?\')", r'<span class="code-string">\1</span>', code)
        
        code = '\n'.join([f'<span class="code-comment">{line}</span>' if line.strip().startswith('#') else line for line in code.split('\n')])

        pattern = re.compile(r'\b(' + '|'.join(re.escape(k) for k in keywords) + r')\b|\b\d+\.?\d*\b')
        code = pattern.sub(replace_with_span, code)
        
        return code

# ======================================================================
# 7. UI AND MAIN APPLICATION LOGIC (FIXED)
# ======================================================================

class BacktesterUI:
    def __init__(self, master):
        self.master = master
        master.title("AlgoHaus Backtester v5.0")
        
        # Default settings
        self.data_folder = pathlib.Path.cwd() / "data" 
        self.df = None
        self.selected_pair = tk.StringVar(master, value="EUR/USD")
        self.selected_timeframe = tk.StringVar(master, value="1hr")
        self.selected_strategy = tk.StringVar(master, value="vwap_crossover_strategy")
        self.initial_balance = tk.DoubleVar(master, value=10000.0)
        self.unit_size = tk.IntVar(master, value=10000)
        self.leverage = tk.IntVar(master, value=50)
        self.sl_pips = tk.IntVar(master, value=30)
        self.tp_pips = tk.IntVar(master, value=60)
        self.risk_percent = tk.DoubleVar(master, value=1.0)
        
        # Date variables
        today = date.today()
        self.end_date_var = tk.StringVar(master, value=today.strftime("%Y-%m-%d"))
        self.start_date_var = tk.StringVar(master, value=(today - timedelta(days=365)).strftime("%Y-%m-%d"))
        
        # Status/Result variables
        self.status_text = tk.StringVar(master, value="Ready.")
        self.metrics_data = {}
        self.trades_df = pd.DataFrame() # <--- FIX 1: Initialize trades_df storage
        
        self.setup_ui()

    def setup_ui(self):
        # (UI setup code remains the same as previous version)
        
        # Frame for controls
        control_frame = ttk.Frame(self.master, padding="10")
        control_frame.grid(row=0, column=0, sticky="nsew")

        # Configuration Section
        config_frame = ttk.LabelFrame(control_frame, text="⚙️ Configuration", padding="10")
        config_frame.grid(row=0, column=0, padx=10, pady=10, sticky="ew")

        # Data Folder
        ttk.Label(config_frame, text="Data Folder:").grid(row=0, column=0, sticky="w", pady=2)
        self.folder_label = ttk.Label(config_frame, text=str(self.data_folder), foreground="blue")
        self.folder_label.grid(row=0, column=1, sticky="w", pady=2)
        ttk.Button(config_frame, text="Change", command=self.select_data_folder).grid(row=0, column=2, padx=5)

        # Pair Selection
        ttk.Label(config_frame, text="Trading Pair:").grid(row=1, column=0, sticky="w", pady=2)
        pairs = list(ForexCalculator.PIP_VALUES.keys())
        ttk.Combobox(config_frame, textvariable=self.selected_pair, values=pairs, state="readonly").grid(row=1, column=1, sticky="ew", pady=2)

        # Timeframe
        ttk.Label(config_frame, text="Timeframe:").grid(row=2, column=0, sticky="w", pady=2)
        timeframes = ["1min", "5min", "15min", "1hr", "1Day"]
        ttk.Combobox(config_frame, textvariable=self.selected_timeframe, values=timeframes, state="readonly").grid(row=2, column=1, sticky="ew", pady=2)

        # Date Range
        ttk.Label(config_frame, text="Start Date (YYYY-MM-DD):").grid(row=3, column=0, sticky="w", pady=2)
        ttk.Entry(config_frame, textvariable=self.start_date_var).grid(row=3, column=1, sticky="ew", pady=2)
        ttk.Label(config_frame, text="End Date (YYYY-MM-DD):").grid(row=4, column=0, sticky="w", pady=2)
        ttk.Entry(config_frame, textvariable=self.end_date_var).grid(row=4, column=1, sticky="ew", pady=2)

        # Strategy Section
        strategy_frame = ttk.LabelFrame(control_frame, text="💡 Strategy & Risk", padding="10")
        strategy_frame.grid(row=1, column=0, padx=10, pady=10, sticky="ew")
        
        # Strategy Selection
        ttk.Label(strategy_frame, text="Strategy:").grid(row=0, column=0, sticky="w", pady=2)
        strategies = [name for name, obj in inspect.getmembers(TradingStrategies) if inspect.isfunction(obj)]
        ttk.Combobox(strategy_frame, textvariable=self.selected_strategy, values=strategies, state="readonly").grid(row=0, column=1, sticky="ew", pady=2)

        # Risk parameters
        ttk.Label(strategy_frame, text="Stop Loss (Pips):").grid(row=1, column=0, sticky="w", pady=2)
        ttk.Entry(strategy_frame, textvariable=self.sl_pips).grid(row=1, column=1, sticky="ew", pady=2)
        ttk.Label(strategy_frame, text="Take Profit (Pips):").grid(row=2, column=0, sticky="w", pady=2)
        ttk.Entry(strategy_frame, textvariable=self.tp_pips).grid(row=2, column=1, sticky="ew", pady=2)

        # Account Section
        account_frame = ttk.LabelFrame(control_frame, text="🏦 Account Settings", padding="10")
        account_frame.grid(row=2, column=0, padx=10, pady=10, sticky="ew")
        
        ttk.Label(account_frame, text="Initial Balance ($):").grid(row=0, column=0, sticky="w", pady=2)
        ttk.Entry(account_frame, textvariable=self.initial_balance).grid(row=0, column=1, sticky="ew", pady=2)
        
        ttk.Label(account_frame, text="Unit Size (Fixed):").grid(row=1, column=0, sticky="w", pady=2)
        ttk.Entry(account_frame, textvariable=self.unit_size).grid(row=1, column=1, sticky="ew", pady=2)

        ttk.Label(account_frame, text="Leverage (e.g., 50):").grid(row=2, column=0, sticky="w", pady=2)
        ttk.Combobox(account_frame, textvariable=self.leverage, values=ForexCalculator.LEVERAGE_OPTIONS, state="readonly").grid(row=2, column=1, sticky="ew", pady=2)

        # Run Button
        ttk.Button(control_frame, text="🚀 Run Backtest", command=self.start_backtest_thread).grid(row=3, column=0, padx=10, pady=10, sticky="ew")

        # Results Frame
        results_frame = ttk.Frame(self.master, padding="10")
        results_frame.grid(row=0, column=1, sticky="nsew")
        self.master.grid_columnconfigure(1, weight=1)
        self.master.grid_rowconfigure(0, weight=1)

        # Summary Display
        summary_label_frame = ttk.LabelFrame(results_frame, text="📈 Summary", padding="10")
        summary_label_frame.grid(row=0, column=0, sticky="ew", pady=10)
        self.summary_display = scrolledtext.ScrolledText(summary_label_frame, width=50, height=8, font=('Consolas', 10), background='#222', foreground='#eee')
        self.summary_display.pack(fill="both", expand=True)

        # Metrics Display (Treeview)
        metrics_label_frame = ttk.LabelFrame(results_frame, text="📊 Detailed Metrics", padding="10")
        metrics_label_frame.grid(row=1, column=0, sticky="nsew", pady=10)
        
        self.metrics_tree = ttk.Treeview(metrics_label_frame, columns=('Key', 'Value'), show='headings', height=20)
        self.metrics_tree.heading('Key', text='Metric')
        self.metrics_tree.heading('Value', text='Value')
        self.metrics_tree.column('Key', width=150, anchor='w')
        self.metrics_tree.column('Value', width=150, anchor='e')
        self.metrics_tree.pack(fill="both", expand=True)
        
        # Report Button
        ttk.Button(results_frame, text="📋 Generate HTML Report", command=self.generate_report).grid(row=2, column=0, pady=10, sticky="ew")

        # Status Bar
        status_bar = ttk.Frame(self.master, relief=tk.SUNKEN, borderwidth=1)
        status_bar.grid(row=1, column=0, columnspan=2, sticky="ew")
        ttk.Label(status_bar, textvariable=self.status_text, anchor="w", padding="2").pack(fill="x")


    def select_data_folder(self):
        # (Method remains the same as previous version)
        new_folder = filedialog.askdirectory(title="Select Data Folder")
        if new_folder:
            self.data_folder = pathlib.Path(new_folder)
            self.folder_label.config(text=str(self.data_folder))
            self.status_text.set(f"Data folder set to: {self.data_folder}")
            
    def start_backtest_thread(self):
        # (Method remains the same as previous version)
        self.status_text.set("Loading data and starting backtest...")
        
        # Clear previous results
        self.summary_display.delete('1.0', tk.END)
        self.trades_df = pd.DataFrame() # Clear old trades
        for item in self.metrics_tree.get_children():
            self.metrics_tree.delete(item)
            
        # Validate inputs
        try:
            start_date = datetime.strptime(self.start_date_var.get(), "%Y-%m-%d")
            end_date = datetime.strptime(self.end_date_var.get(), "%Y-%m-%d")
            sl_pips = self.sl_pips.get()
            tp_pips = self.tp_pips.get()
            balance = self.initial_balance.get()
            units = self.unit_size.get()
            leverage = self.leverage.get()
            
            if start_date >= end_date:
                raise ValueError("Start date must be before End date.")
            if sl_pips <= 0 or tp_pips <= 0 or balance <= 0 or units <= 0 or leverage <= 0:
                raise ValueError("SL, TP, Balance, Unit Size, and Leverage must be positive numbers.")

        except ValueError as e:
            messagebox.showerror("Input Error", f"Invalid input: {e}")
            self.status_text.set("Error: Invalid input.")
            return

        # Start the background thread for backtesting
        self.q = queue.Queue()
        threading.Thread(target=self.run_backtest_task, args=(start_date, end_date), daemon=True).start()
        self.master.after(100, self.check_queue)
        
    def run_backtest_task(self, start_date, end_date):
        # (Method remains the same as previous version)
        try:
            pair = self.selected_pair.get()
            timeframe = self.selected_timeframe.get()
            strategy_name = self.selected_strategy.get()
            
            # 1. Load Data
            df, _, _ = load_pair_data(pair, self.data_folder, start_date, end_date, timeframe)
            self.df = df
            
            # 2. Get Strategy Function
            strategy_func = getattr(TradingStrategies, strategy_name)
            
            # 3. Initialize and Run Backtester
            backtester = EnhancedBacktester(
                df, 
                initial_balance=self.initial_balance.get(), 
                unit_size=self.unit_size.get(), 
                pip_value=ForexCalculator.PIP_VALUES.get(pair, 0.0001), 
                leverage=self.leverage.get()
            )
            
            summary, metrics = backtester.run_backtest(
                strategy_func, 
                self.sl_pips.get(), 
                self.tp_pips.get(), 
                pair
            )
            
            # Place results in the queue
            self.q.put(('success', summary, metrics, backtester.results))

        except Exception as e:
            logging.error(f"Backtest Thread Error: {e}")
            self.q.put(('error', str(e)))

    def check_queue(self):
        # (Method remains the same as previous version)
        try:
            result_type, *data = self.q.get_nowait()
            
            if result_type == 'success':
                summary, metrics, trades_df = data
                self.update_results_ui(summary, metrics, trades_df)
                self.status_text.set("Backtest finished successfully.")
            elif result_type == 'error':
                error_msg = data[0]
                messagebox.showerror("Backtest Error", error_msg)
                self.status_text.set("Error: Backtest failed.")

        except queue.Empty:
            # Check again soon if the queue is still empty
            self.master.after(100, self.check_queue)
            
    def update_results_ui(self, summary, metrics, trades_df):
        # --- FIX 2: Save the trades DataFrame to the class instance ---
        self.trades_df = trades_df 
        
        # Update Summary
        self.summary_display.delete('1.0', tk.END)
        self.summary_display.insert(tk.END, summary)
        
        # Update Metrics Treeview
        self.metrics_data = metrics
        for item in self.metrics_tree.get_children():
            self.metrics_tree.delete(item)
            
        for key, value in metrics.items():
            if isinstance(value, float):
                value_str = f"{value:,.2f}"
            else:
                value_str = str(value)
            
            tag = 'positive' if 'win' in key.lower() or 'return' in key.lower() or 'profit' in key.lower() and value > 0 else 'negative'
            self.metrics_tree.insert('', tk.END, values=(key.replace('_', ' ').title(), value_str), tags=(tag,))
        
        self.metrics_tree.tag_configure('positive', foreground='green')
        self.metrics_tree.tag_configure('negative', foreground='red')

    def generate_report(self):
        # --- FIX 3: Use the stored trades_df for the report ---
        if self.trades_df.empty:
            messagebox.showwarning("Report Warning", "No trades found. Please run a successful backtest first.")
            return

        try:
            # Pass self.trades_df, which was saved in update_results_ui
            report_path = AdvancedHTMLReportGenerator.generate_dashboard_report(
                self.metrics_data, 
                self.trades_df, 
                getattr(TradingStrategies, self.selected_strategy.get()), 
                self.selected_timeframe.get(), 
                self.selected_pair.get(), 
                self.initial_balance.get(),
                self.unit_size.get(),
                df=self.df # Pass the loaded OHLC data for charting
            )
            
            messagebox.showinfo("Report Generated", f"HTML Report saved to: {report_path}")
            webbrowser.open_new_tab('file://' + os.path.realpath(report_path))
            self.status_text.set("HTML report generated and opened.")
        
        except Exception as e:
            messagebox.showerror("Report Error", f"Failed to generate report: {e}")
            self.status_text.set("Error generating report.")

if __name__ == '__main__':
    root = tk.Tk()
    app = BacktesterUI(root)
    root.mainloop()

ERROR: Backtest Thread Error: No CSV found for EUR/USD
C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_5800\363038480.py:243: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample(rule).agg({
INFO: Dataframe size: 6263 bars.
INFO: Strategy generated: 1269 potential trades.
c:\Users\Wolfrank\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\Wolfrank\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain t